# Topic analysis based on Genius songs lyrics

Author : lievre.thomas@gmail.com

---

In this notebook, we will explore the genius data extract from [Kaggle](https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information).

**The aim of this analysis is to retrieve topic from lyrics and retrieve main topics by year or decade.**

This notebook was carried out in the context of a class project imposed by the [text mining course (TDDE16)](https://www.ida.liu.se/~TDDE16/project.en.shtml) of Linköpings universitet.


## Few informations about Genius website

Genius is an American digital company founded on August 27, 2009, by Tom Lehman, lan Zechory, and Mahbod Moghadam. Originally launched as Rap Genius with a focus on hip-hop music, it was initially a crowdsourced website where people could fill in the lyrics of rap music and give an interpretation of the lyrics. Over the years the site has grown to contain several million annotated texts from all eras ( from [Wikipedia Genius page](https://en.wikipedia.org/wiki/Genius_(company))).


## Load the data in memory

Data are all contain in a big 9GB csv file (around 5 millions rows). It could be dificult to load all this data in our computer memory. To deal with this issue, I made a loading class to split the data in 6 pickles files to improve the compressness of the data which aim to improve the loading speed in the memory. Then the pickles are randomly draw to improve generality of the data. We currently assumed the data pickles batch are identically distributed (we will explore the data batches at the second part). The class below deal with all the process.

In [1]:
import pandas as pd
from random import seed, sample
import pickle
import glob
import os

class Loader():

    def __init__(self, in_path, out_path):
        """
        Args:
            in_path (str): csv input path.
            out_path (str): Output directory path to store the pickles.
            chunksize (int, optional): Chunksize for DataFrame reader. Defaults to 10**6. 
        """

        self.__in_path = in_path
        self.__out_path = out_path
        self.__chunksize = 10**6

    def __produce_pickles(self):
        """produce pickles by reading csv by chunksize
        """
        with pd.read_csv(self.__in_path, chunksize = self.__chunksize) as reader:
            try:
                os.makedirs(self.__out_path)
            except FileExistsError:
                # directory already exists
                pass
            for i, chunk in enumerate(reader):
                out_file = self.__out_path + "/data_{}.pkl".format(i+1)
                with open(out_file, "wb") as f:
                    pickle.dump(chunk, f, pickle.HIGHEST_PROTOCOL)
    
    def load_pickle(self, pickle_id):
        """load a pickle file by id

        Args:
            pickle_id (int): pickle id.

        Raises:
            Exception: The path of the given id isn't a file

        Returns:
            obj: DataFrame
        """
        # produce the pickles if the directory not exists or
        # if the directory is empty 
        if (not os.path.exists(self.__out_path)) or \
              (len(os.listdir(self.__out_path)) == 0):
            self.__produce_pickles()
        
        # get the file path following the pickle_id
        # given in parameter
        file_path = self.__out_path + \
            "/data_" + str(pickle_id) + ".pkl"

        if os.path.isfile(file_path):
            df = pd.read_pickle(file_path)
        else:
            raise Exception("The pickle file data_{}.pkl doesn't exist".format(pickle_id))
        return df
        

    def random_pickles(self, n_pickles = 3, init = 42, verbose = True):
        """random reader over pickles files

        Args:
            n_pickles (int, optional): number of pickles to load. Defaults to 3.
            init (int, optional): Integer given to the random seed. Defaults to 42.
            verbose (bool, optional): Print the loaded files. Defaults to True

        Raises:
            Exception: Stop the process if n_pickles exceed pickle files number.

        Returns:
            obj: pd.Dataframe
        """

        # produce the pickles if the directory not exists or
        # if the directory is empty 
        if (not os.path.exists(self.__out_path)) or \
              (len(os.listdir(self.__out_path)) == 0):
            self.__produce_pickles()

        pickle_files = [name for name in
                        glob.glob(self.__out_path + "/data_*.pkl")]
        # draw p_files        
        seed(init)

        if n_pickles <= 6:
            random_p_files = sample(pickle_files, n_pickles)
        else:
            raise Exception("The parameter n_pickles (" +
                            "{}) exceed the numbers of pickle files ({})"\
                                .format(n_pickles, len(pickle_files)))
        # print the drawed files
        if verbose:
            print("Loaded pickles:")
            for p in random_p_files:
                print(p)

        # load random pickles file
        df_list = [pd.read_pickle(p) for p in random_p_files]

        # create the dataframe by concatenate the previous
        # dataframes list
        df = pd.concat(df_list, ignore_index = True)
        return df

In [2]:
# create reader
#  /!\ change path in kaggle
kaggle_input = "/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv"
kaggle_output = "/kaggle/working/data/"

# initiate the file loader
loader = Loader(in_path = kaggle_input, out_path = kaggle_output)

# load random pickle files
df = loader.random_pickles(n_pickles = 1)

Loaded pickles:
/kaggle/working/data/data_3.pkl


Batchs of data are randomly loaded in the memory. The number of batchs loaded depends on the memory capacity of the computer running the script. For the analysis, we will only works on the random samples loaded (All the data in Kaggle).  

## Exploring the coarse data

Let's visualize and explore the coarse data before a part of deeper analysis.

In [3]:
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Roses,rock,Vitja,2017,399,{},The roses start to wither\nWhere the devil lay...,3019113,en,en,en
1,Keep On Pushin,rap,Problem,2017,692,"{""My Princess Aeryn""}",[Hook]\nImma keep on pushin\nImma keep on push...,3019114,en,en,en
2,Inside,pop,The jepettos,2017,1302,{},"[Intro]\nOoh, ooh, ah, ah (x2)\n\n[Verse 1]\nI...",3019115,en,en,en
3,Girls Like You,rb,PnB Rock,2017,60114,{},[Chorus]\nBaby it was real\nYeah we were the b...,3019117,en,en,en
4,Froideur,rap,N'Dirty Deh,2017,5813,"{""N\\'Dirty Deh""}","[Refrain]\nEt j'ai perdu la foi, mais c'est pa...",3019120,fr,fr,fr


For each songs, we've got several informations :
- title of the song
- the tag (which kind of music)
- the artist singer name
- the release year
- the number of page views
- the featuring artists names
- the lyrics
- the genius identifier
- Lyrics language according to [CLD3](https://github.com/google/cld3). Not reliable results are NaN. CLD3 is a neural network model for language indentification.
- Lyrics language according to [FastText's langid](https://fasttext.cc/docs/en/language-identification.html). Values with low confidence (<0.5) are NaN. FastText's langid is library developped by Facebook’s AI Research lab for efficient learning of word representations and sentence classification. fastText has also published a fast and accurate tool for text-based language identification capable of recognizing more than 170 languages.
- Combines language_cld3 and language_ft. Only has a non NaN entry if they both "agree".

More information at this link : https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information

In [4]:
df.dtypes

title            object
tag              object
artist           object
year              int64
views             int64
features         object
lyrics           object
id                int64
language_cld3    object
language_ft      object
language         object
dtype: object

In [5]:
# display the size
print('Data frame size (row x columns):', df.size)
print('Data rows number: ', len(df))
print('Number of unique songs (following genius id): ', len(df.id.unique()))

Data frame size (row x columns): 11000000
Data rows number:  1000000
Number of unique songs (following genius id):  1000000


Genius id seems to be the unique rows identifier.

Let's vizualise size of the coarse data over years before preprocessing to compare batch distributions. One things to know before vizualise the data, the pickles are create by chunks reading. 

The last diplayed table gives us some information about the data. The csv file seems to be sort by id, so the pickle files are then sort too.

In [6]:
import plotly.express as px
import plotly.io as pio

# change theme template for every graph below
pio.templates.default = "plotly_white"


# get some information about the pickle data
def pickle_informations(data = loader):
    rows = []
    for i in range(1, len(os.listdir('data')) + 1):
        df = data.load_pickle(i)
        rows.append(len(df))
        del df
    return rows

# get the rows
rows = pickle_informations()

# create the dataframe
df_data = pd.DataFrame(
    {'batch' : ['data ' + str(i) for i in range(1,len(rows) + 1)],
    'rows' : rows})

fig = px.bar(df_data, x="batch", y="rows")
fig.show()

Batch seems to have the same number of rows rexcept for the last one which is consistent because batch are create iteratively by 10e6 chunks over the csv The last batch could be seen as a rest.

In [7]:
import plotly.graph_objects as go

def add_bar(i, y1, y2, color, data = loader):
    df = data.load_pickle(i)
    df = df[(df.year >= y1) & (df.year <= y2)]
    df_year = df.groupby(['year']).size().reset_index(name='count')
    new_bar = go.Bar(
                x = df_year.year.values,
                y = df_year['count'].values,
                name = 'data_'+ str(i),
                marker = {'color' : color})
    new_trend = go.Scatter(
                x = df_year.year.values,
                y = df_year['count'].values,
                mode="lines",
                line={'color' : color,
                    'width' : 0.5},
                showlegend=False)
    del df_year, df
    return new_bar, new_trend


def multi_barplot(year1, year2, colors):    
    # create a empty plotly.Figure object
    fig = go.Figure() 
    # compute the batch number
    n_batch = len(os.listdir('data'))
    # test the color list feed in argument
    # fit well with the batch number
    if n_batch > len(colors):
        raise Exception(
            "The colors list size({})doesn't ".format(len(colors)) +
            "fit with the number of data".format(n_batch))
    for i in range(1, n_batch + 1):
        fig.add_traces((add_bar(i, year1, year2, colors[i-1])))
    fig.update_layout(
        title = "Data distribution over years ({} - {})"
            .format(year1, year2),
        xaxis_title="years",
        yaxis_title="title",
        legend_title="Data batch")
    return fig


In [8]:
import plotly.colors as col

# create the color list
colors = col.qualitative.Plotly

# 1990 - 2023
fig1 = multi_barplot(1960, 1989, colors)
fig1.show()
# 1960 - 1990
fig2 = multi_barplot(1990, 2023, colors)
fig2.show()

The first bar chart (1960 - 1989) shows an increasing numbers of data over years. Moreover batch seems to have quite similar distriutions over years. data_1 and data_2 batch quite outperform the 4 others. data_6 batch is weaker than the other due to its poor number of rows.
The data behaves similarly until 2012 as we can see on the second chart (1990-2023). After this year there is great increasing of the data retrieved. A minimum increase of at least 100% of the batch can be observed. An increase of up to 50 times the batch size for some.

## Data pre-processing

The aim of this part is to preprocess data in order to get suitable data for the analysis. let's focus on the year variable.

We will focus on English songs, to facilitate the analysis and the work of natural language processing algorithms.

In [9]:
# Retrieve only the texts identified as English language by both cld3 and fasttext langid
df = df[df.language == 'en']

Next, it can be quite interseting to check Nan values

In [10]:
# find which column contain nan value
df.columns[df.isna().any()].tolist()

['title']

In [11]:
# get all rows that contain NaN values
df_nan = df[df.isna().any(axis=1)]
df_nan

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
114245,NaN,rock,Peter Dzubay,2017,52,{},"When I finally find what's real, why shouldn't...",3166588,en,en,en
146772,NaN,rap,OLlama,2017,2120,{},[Intro]\nToo many bitches; Too many bitches tr...,3210807,en,en,en
158344,NaN,rock,The Moth & The Flame,2011,1051,{},We used to be so similar\nWell that was wishfu...,3226446,en,en,en
160846,NaN,misc,Lawfermz,2017,3,{},(Intro)\nBeep beep beep beep beep beep.\nHello...,3230118,en,en,en
210243,NaN,rap,TripleYoThreat,2017,24,{},"TripleYoThreat, you ain't never gonna make it\...",3296318,en,en,en
211581,NaN,rap,Huntaps,2017,157,{},"[Hook]\nFuck 12, if you fuck with me I'll send...",3297867,en,en,en
294150,NaN,rap,Shaiza Maponyaza,2018,45,{},[intro]\n\nYah yah yah yah\nYah yah\nMan same ...,3438103,en,en,en
309207,NaN,rock,PAWS,2016,271,{},You're scared of history repeating on me\nCons...,3472732,en,en,en
357786,NaN,rap,J4ydizz1e,2018,55,{},[Intro: J4yDiZz1e]\n\nThis shit sounds just li...,3561848,en,en,en
441948,NaN,rap,Bass Santana,2018,1147,"{""Kin\\$oul"",""Skott Summerz""}",[Intro: Bass Santana]\nTo Bass be the glory\nF...,3681585,en,en,en


In [12]:
print('Number of untitled song:', len(df[df.isna().any(axis=1)]))

Number of untitled song: 21


Insofar as the title of the music is not to be taken into account in the learning of the topic modeling algorithms but But the titles can be related to the topics in the next phase of analysis and the low number of songs without any title, I decide to delete this data for the moment.

In [13]:
# Delete rows containing NaN values
df = df.dropna()
len(df)

645573

Next, we also try to check for None values

In [14]:
df[df.isnull().any(axis=1)]

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language


No None values in this dataframe.

Afterwards, let's look at the year variable, which is one of the important variables to take into account in our analysis because we want to extract the topics by decades.

In [15]:
years = df.year.unique()
print(years)

print('Number of unique years: ',len(years))

[2017 2016 1998 2002 2011 2013 2014 2015 2006 2009 2012 2005 2018 2008
 2003 2010 2007 1989 1986 1999 1905 1985 1994 2001 1899 1913 2000 2020
 1990 1984 1983 1979 2021 1995 2004 1825 1992 1993 2019 1980 1977 1966
 1988 1991 1997 1949 1957 1982 1674 1996 1974 1942 1850 1842    1 1964
 1972 1915 1976 1930 1987 1919 1911 1960 1978 1968 1975 1950 1981 1952
 1973 1951 1947 1954 1961 1892 1878 1934 1935 1970 1969 1944 1962 1909
 2022 1971 1927 1945 1936 1926 1938 1901   12 1900 1922 1929 1937 1953
 1820 1864 1965 1955 1540 1916 1963 1967 1918 1904 1943   14 1772 1788
 1780 1959 1886 1939 1871 1912 1956 1958 1880 1923 1910 1844 1897 1914
 1862 1933 1946   15 1877 1771   17 1948 1931 1898 1700 1888 1928  510
 1791 1895 1941 1920   25 1917 1908 1940 1859 1907 1902 2023 1857 1863
 1666 1893 1889 1861 1869 1925 1867 1872 1676 1675  420 1932  709 1818
 1830 1005 1866 1066  176 1300 1924 1251 1838 1760 1739   21 1758 1469
    2 1400 1853 1320 1200 1890 1860 1903   18 1220  130 1794 1896 1461
 1210 

We firstly want to know if the year variable format is suitable. It is highly likely that year are sometimes downsized (example : 92 instead of 1992).
Let's display the tag distribution for music with a release year below 215.

In [16]:
df_tag = df[df['year'] < 215].groupby(['tag']).size().reset_index(name='count')

fig = px.pie(df_tag, names="tag", values="count", title = "Outlier tag distribution")
fig.show()

It is rather surprising to observe that the majority style of music of this period (< 215) is rap music knowing that this style is known for the current emerging style. Of course, among this data their is a important part of outlier year.

In [17]:
# Extract the pieces of music of type 'rap' lower than the year 215
df_rap = df[(df['year'] < 215) & (df['tag'] == 'rap')]
df_rap.sort_values(by='views',ascending=False).head(20)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
111689,Issa Snack,rap,Phatboitheceleb,1,4182,{},Chorus: its ya birthday and you know I ain't f...,3163216,en,en,en
268205,Oof sandman sans sans diss track,rap,Squeaky,15,3206,{},OOF POOF BOOTH LOSE LOOSE MOVE LUKE FRISK WRIS...,3387617,en,en,en
218600,First Day Home,rap,RetcH,1,2334,{},"[Intro]\n\n[Chorus]\nHome, it's my first day h...",3307004,en,en,en
381769,Papi,rap,DON-DON (US),1,2129,{},[INTRO]\nCall me papi\nCall me papi\nLead her\...,3596060,en,en,en
715072,Same Love,rap,Macklemore,1,1767,"{""Macklemore & Ryan Lewis""}",When I was in the 3rd grade\nI thought that I ...,4080328,en,en,en
969927,IWroteThisForTaylorSwift,rap,Vershad,1,1607,"{""Brooklyn Zoo""}","[Intro]\nJake: Sounds like your sad. Listen, ...",4484444,en,en,en
268744,I like frisk,rap,Squeaky,1,1186,"{""Squeaky jr""}","Ok, ok, ok, ok, ok, ok, ok, yeah\nI like frisk...",3388825,en,en,en
945081,Same Lil Bitch,rap,Shayla Gessler,1,1023,{},"Did me dirty, you been fuckin' with the Same L...",4441966,en,en,en
18947,Dont Blame Me For You,rap,Kids These Days,1,868,"{""VIC MENSA"",""Macie Stewart"",""Liam Cunningham""}","[Verse 1: Vic Mensa]\nI'm larger than life, ha...",3044399,en,en,en
431118,Robocop,rap,videogamedunkey,1,766,{​videogamedunkey},Now that I'm a Robocop\n\nNow I'm never gonna ...,3666068,en,en,en


In [18]:
df_rap[df_rap['artist'] == 'Kanye East']

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language


If we search the release date of this track on google, we can find a release date from 4 May 2021 on the [Genius website](https://genius.com/Kanye-east-the-secrets-of-dababy-lyrics). Given the year that we find in our table and real one, we can assume some issue about the date format (1 instead 2021).

After few research on genius website, the most viewed songs of this above displayed list seems to be released on 2021 but more views decrease harder is the interpretation of date.

Let's check the second most popular tag 'pop' in this retrieve outliers data :

In [19]:
# Extract the pieces of music of type 'rap' lower than the year 215
df_pop = df[(df['year'] < 215) & (df['tag'] == 'pop')]
df_pop.sort_values(by='views',ascending=False).head(20)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
274779,Off White,pop,Zach Clayton,1,7894,{},"Chorus\nSwish on my kicks, off white\nStripes ...",3399877,en,en,en
951623,I dont wanna see you cryin anymore,pop,Adam Melchor,1,7479,{},[Verse 1]\nI don't wanna see you cryin’ anymor...,4452665,en,en,en
495830,Hello from the Dark Side,pop,RoyishGoodLooks,15,7019,{},[Chorus]\nHello from the Dark Side\nI must've ...,3763085,en,en,en
385856,The Moons Detriment,pop,Shannon Lay,1,6698,{},If I were to know you\nI’d show you all the li...,3603840,en,en,en
635909,Mostly to Yourself,pop,Noah Reid,1,3517,{},Well it's mostly in the mornin'\nWhen your eye...,3961929,en,en,en
433617,Come September,pop,Anas Mitchell,1,1244,"{""Anaïs Mitchell""}","Autumn's ashes, summer's embers\nOn the sidewa...",3669873,en,en,en
599147,Gone,pop,Ella Martine,1,946,{},[Verse 1]\nIt's hard we hurt each other\nTime ...,3910616,en,en,en
433630,The Pursewarden Affair,pop,Anas Mitchell,1,732,"{""Anaïs Mitchell""}",Percy Pursewarden\nOpen up your door\nI haven'...,3669885,en,en,en
318347,The Rose,pop,Common Holly,1,642,{},Well I know I was the rose\nBut now I feel lik...,3496110,en,en,en
918072,U cut me off so the cuts r on me now..,pop,Sapphire2001,1,495,{},I can feel in the air tonight\nSmoke seeping f...,4401036,en,en,en


The titles recovered seem to be for the most part recent sounds, not very popular with a bad indexation of the years.

A case-by-case pre-processing of the data is too tedious compared to the amount of data to be processed. We will only use data with correctly formatted dates.

In [20]:
df = df[(df.year >= 1960) & (df.year < 2023)]
len(df)

642975

We wish to analyze the texts by decade then let's add a decade column.

In [21]:
import math

df['decade'] = df['year'].map(lambda x : int(math.trunc(x / 10) * 10))

df.sort_values(by = 'year').head(20)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,decade
329044,Darling If I Had You,rock,Bobby Day,1960,308,{},"Darling if I had you, if I had you\nMmh, what ...",3515784,en,en,en,1960
734933,Tonights All Right For Love Bonus Track Version,pop,Elvis Presley,1960,142,{},Hold me tight the moon's so bright\nTonight th...,4111118,en,en,en,1960
752292,Row Your Boat,pop,The Chipmunks,1960,261,{},"Careful now, fellas, steady\nDon't tip it over...",4136301,en,en,en,1960
374556,Two Views of a Cadaver Room,misc,Sylvia Plath,1960,1079,{},(1)\nThe day she visited the dissecting room\n...,3585530,en,en,en,1960
374557,The Eye-Mote,misc,Sylvia Plath,1960,453,{},Blameless as daylight I stood looking\nAt a fi...,3585531,en,en,en,1960
7170,Extra Sensory Perception,pop,Sheldon Allman,1960,193,{},We've got extra sensory perception\nIt makes o...,3029810,en,en,en,1960
374558,Hardcastle Crags,misc,Sylvia Plath,1960,266,{},"Flintlike, her feet struck\nSuch a racket of e...",3585532,en,en,en,1960
374559,Faun,misc,Sylvia Plath,1960,329,{},"Haunched like a faun, he hooed\nFrom grove of ...",3585533,en,en,en,1960
479255,Whirlwind,rb,Barrett Strong,1960,109,"{""The Rayber Voices""}",You know it hit me (Hit me like whirlwind)\nYo...,3738355,en,en,en,1960
374560,Departure,misc,Sylvia Plath,1960,375,{},The figs on the fig tree in the yard are green...,3585534,en,en,en,1960


## Data vizualisation

Let's do some vizualisation to get a better understanding of our data. As we saw on previous distribution graphs over years, more titles have been recorded over the last 2 decades.

In [22]:
# barplot by decade
def barplot_by_decade(df):

    # groupby decade
    df_d = df.groupby(['decade']).size().reset_index(name='count')

    # create the figure
    fig = go.Figure()

    fig.add_bar(
        x=df_d.decade,
        y=df_d['count'],
        showlegend=False)

    fig.add_scatter(
            x=df_d.decade,
            y=df_d["count"],
            mode="markers+lines",
            name="trend",
            showlegend=False)

    fig.update_layout(
            title = "Music release over years",
            xaxis_title="decade",
            yaxis_title="release")
    return fig

# build and display
fig = barplot_by_decade(df)
fig.show()

In [23]:
# compute tag frequencies by decade
df_pies_d = df.groupby(['decade','tag']).size().reset_index(name='count')
df_pies_d[df_pies_d.decade == 1960]

,decade,tag,count
0,1960,country,764
1,1960,misc,271
2,1960,pop,2292
3,1960,rap,8
4,1960,rb,704
5,1960,rock,688


In [24]:
from plotly.subplots import make_subplots

# create en make subplot
fig = make_subplots(rows=3, cols=3,
                    specs=[
                        [{'type':'domain'}
                        for i in range(1,4)] for i in range(1,4)
                    ])
decades = df_pies_d.decade.unique().tolist()
for i in range(0,3):
    for k in range(0,3):
        decade = decades[i*3 + k]
        # group by decade
        df_p = df_pies_d[df_pies_d.decade == decade]
        # add figure
        fig.add_trace(go.Pie(labels=df_p.tag, values=df_p['count'], name=decade), i+1, k+1)
        # add annotation
        fig.add_annotation(arg=dict(
            text=decade, x=k*0.375 + 0.125,
            y=-i*0.3927 + 0.90, font_size=10,
            showarrow=False))
        if (i*3 + k) == 6:
            break


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Tags proportions by decades"
    # Add annotations in the center of the donut pies.
    #annotations=[dict(text=decade, x=k*0.375+0.125, y= -i*0.125+0.90, font_size=10, showarrow=False)
     #           for k, decade in enumerate(decades) for i in range(0,4)]
)
fig.show()

The different pie charts show us an evolution of the different proportions of music styles by decades from 1960 to today. In the sixties the most listed music style is the **pop music**. Then comes an emergence of a rising style: the **rock**. This one increases with the decades until it overtakes the pop music during the decades 90 and 2000. The two styles come to balance thereafter facing the meteoric rise of the most listed genre in our current decade, namely **rap**. The evolution of the proportions recover can give us a rather precise idea of the most popular styles of their times. However, it is important to remember that the data still has some bias, Genius being a crowdsourced tool created during the last decade and at the base only as a lexical translator of rap music, it is normal to find a large amount of data from this period and especially from this style there.

## Text preprocessing

After the visualisation part let's focus more on the main data which are the lyrics.

In [25]:
df.iloc[0]["lyrics"]

"The roses start to wither\nWhere the devil lays to sleep\nAnd sorrow seems to be like joy\nWhere dying lights dance on last time\n\nYou had me at hello\nBut I couldn't see right through your twisted soul\nThis was the downfall of us all\n\n(Roses start to wither\nWhere the devil lays to sleep)\n\nToo close to hell\nI can't see\nWhat ate me up from the inside\nToo close to hell\nI will never see\nWhat ate me up from the inside out\n(From the inside out)\n\nI'm caught in a nightmare\nNo chance to make it out alive\nWhy have we left the place\nWhere we found love\nYou had me at hello\nBut I couldn't see right through your twisted soul\nThis was the downfall of us all\n\n(Roses start to wither\nWhere the devil lays to sleep)\n\nToo close to hell\nI can't see\nWhat ate me up from the inside\nToo close to hell\nI will never see\nWhat ate me up from the inside out\n\nSo sing along\nWe are a battle\nSo sing along\nWhy have you forsaken me?\n\nWhy have you forsaken me?\nWhy have you forsaken m

There is many undesirable characters like the line breaker '\n', figures or square, curly and simple brackets. So let's clean this data with regular expressions.

In [26]:
import re
from numpy.random import randint

def clean_text(text):
    # remove \n
    text = text.replace(r'\n', '')
    # remove punctuation
    text = re.sub(r'[,\.!?]', '', text)
    #removing text in square braquet
    text = re.sub(r'\[.*?\]', ' ', text)
    #removing numbers
    text = re.sub(r'\w*\d\w*',' ', text)
    #removing bracket
    text = re.sub(r'[()]', ' ', text)
    # convert all words in lower case
    text = text.lower()
    return text

# get the results of data cleaning
cleaned_text = df["lyrics"].apply(clean_text)

In [27]:
docs = cleaned_text.to_list()
docs[0]

"the roses start to wither\nwhere the devil lays to sleep\nand sorrow seems to be like joy\nwhere dying lights dance on last time\n\nyou had me at hello\nbut i couldn't see right through your twisted soul\nthis was the downfall of us all\n\n roses start to wither\nwhere the devil lays to sleep \n\ntoo close to hell\ni can't see\nwhat ate me up from the inside\ntoo close to hell\ni will never see\nwhat ate me up from the inside out\n from the inside out \n\ni'm caught in a nightmare\nno chance to make it out alive\nwhy have we left the place\nwhere we found love\nyou had me at hello\nbut i couldn't see right through your twisted soul\nthis was the downfall of us all\n\n roses start to wither\nwhere the devil lays to sleep \n\ntoo close to hell\ni can't see\nwhat ate me up from the inside\ntoo close to hell\ni will never see\nwhat ate me up from the inside out\n\nso sing along\nwe are a battle\nso sing along\nwhy have you forsaken me\n\nwhy have you forsaken me\nwhy have you forsaken me\

In [28]:
# update dataframe
df.update(cleaned_text)
df.head(3)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,decade
0,Roses,rock,Vitja,2017,399,{},the roses start to wither\nwhere the devil lay...,3019113,en,en,en,2010
1,Keep On Pushin,rap,Problem,2017,692,"{""My Princess Aeryn""}",\nimma keep on pushin\nimma keep on pushin\ns...,3019114,en,en,en,2010
2,Inside,pop,The jepettos,2017,1302,{},\nooh ooh ah ah \n\n \ni heard you were th...,3019115,en,en,en,2010


That's better! The libraries that we will use later to perform topic modeling usually provide preprocessing but it is always good to have control over what we manipulate.

## Topic modeling

I will perform 2 ways to do topic modeling :
- [LDA (latent dirichlet allocation)](https://fr.wikipedia.org/wiki/Allocation_de_Dirichlet_latente) are the common way to do topic modeling in the few last years, it works and it's quite easy to use with common python library like [Gensim](https://radimrehurek.com/gensim/auto_examples/index.html).
- [BERTopic](https://maartengr.github.io/BERTopic/index.html) seems to be one of the best technic this day to perform topic modeling. It combine the leverage of [BERT](https://en.wikipedia.org/wiki/BERT_(language_model)) the famous language model with [c-TF-IDF](https://maartengr.github.io/BERTopic/api/ctfidf.html) tansformer. 


### Define default tokenizer and Lemmatizer

In [29]:
import spacy

print("set gpu: ", spacy.prefer_gpu())

# small model /!\ take the bigger one for Kaggle
new_nlp = spacy.load('en_core_web_sm')

set gpu:  False


It could be difficult to process all this data on my computer or Kaggle. The memory will quickly be overwhelmed. I will work with a sample of our previously load data in order to avoid memory overload.

In [30]:
# set sample frac of the data
prop = 0.1

# create sample df 1/3 of the actual loaded data
sdf = df.sample(frac = prop)

In [31]:
# check the distribution of the sample
barplot_by_decade(sdf)

In [32]:
# default preprocessing
def preprocess(text, nlp = new_nlp):

    #TOKENISATION
    tokens =[]
    for token in nlp(text):
        tokens.append(token)

    #REMOVING STOP WORDS
    spacy_stopwords = new_nlp.Defaults.stop_words
    sentence =  [word for word in tokens if word.text.isalpha() and word.text not in spacy_stopwords]

    #LEMMATISATION
    sentence = [word.lemma_ for word in sentence]

    return sentence

### Vizualise most frequent words over decades

In [33]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

class TermsDocumentsMatrix():
    
    def __init__(self, sdf, decades=[1960, 1970], colorscale = 'Plotly3'):
        # vectorizer on the sample lyrics
        self.__vectorizer = CountVectorizer(tokenizer = preprocess)
        # fit and transform the data
        self.__data_vectorized = self.__vectorizer.fit_transform(
            tqdm(sdf['lyrics'].loc[sdf['decade'].isin(decades)])
        )
        # get decades informations
        self.__decades = sdf['decade'].loc[sdf['decade'].isin(decades)].reset_index(drop=True)
        self.__unique_decades = decades
        # get colorscale template
        self.__colorscale = colorscale
    
    def get_tdmatrix(self):
        
        # compute a Matrix terms document by decades
        df_bw = pd.DataFrame(self.__data_vectorized.toarray(),
                    columns = self.__vectorizer.get_feature_names_out())
        
        # check the length
        if len(df_bw) != len(self.__decades):
            raise Exception('Not the same size')
        
        # concatenate decade
        df_bw['decade'] = self.__decades
        
        # check NaN values
        if len(df_bw.columns[df_bw.isna().any()].tolist()) != 0:
            raise Exception('Decade got Nan values')

        return df_bw
    
    def get_tdm_by_decade(self, decade):
        
        if decade not in self.__unique_decades:
            raise Exception("{} doesn't appear in the decades list".format(decade))
        
        # compute a Matrix terms document by decades (bag of words format)
        df_bw = pd.DataFrame(self.__data_vectorized.toarray(),
                    columns = self.__vectorizer.get_feature_names_out())
        
        # check the length
        if len(df_bw) != len(self.__decades):
            raise Exception('Not the same size')
        
        # concatenate decade
        df_bw['decade'] = self.__decades
        
        # check NaN values
        if len(df_bw.columns[df_bw.isna().any()].tolist()) != 0:
            raise Exception('Decade got Nan values')
        
        # select suitable decade
        df_bw = df_bw[df_bw['decade'] == decade]
        
        return df_bw
    
    def most_freq_terms(self, n_rows = 1, n_cols = 2, n_terms = 10):
        
        # create the document terms matrix
        df_bw = self.get_tdmatrix()
        
        # create en make subplot
        fig = make_subplots(rows=n_rows, cols=n_cols,
                            x_title = 'number of occurrences',
                            y_title = 'terms',
                            subplot_titles = self.__unique_decades)
        
        for i in range(0,n_rows):
            for k in range(0,n_cols):
                if (i*n_rows + k) == len(self.__unique_decades):
                    break
                
                # get the decade
                decade = self.__unique_decades[i*n_rows + k]
            
                #select the suitable decade and delete decade column
                df_decade = df_bw.loc[df_bw.decade == decade, df_bw.columns != 'decade']
                
                # compute terms frequencies by decade
                terms_freq = df_decade.sum().sort_values(ascending = False)
            
                # total number of terms occurences
                total_terms = terms_freq.values
                
                # add figure
                fig.add_trace(go.Bar(y=terms_freq.index.tolist()[:n_terms][::-1],
                                     x=total_terms[:n_terms][::-1],
                                     name=decade,
                                     orientation='h', showlegend = False,
                                    marker = dict(color = total_terms,
                                                  colorscale=self.__colorscale)),
                              i+1, k+1)
        return fig

In [34]:
# first decades
tdm = TermsDocumentsMatrix(sdf, decades = [1960, 1970],
                           colorscale = 'Plotly3')

# display bar charts of most frequent terms
tdm.most_freq_terms(n_rows = 1, n_cols = 2, n_terms = 15)

100%|██████████| 1152/1152 [00:39<00:00, 28.98it/s]


According to the bar graphs displayed above, a group of words seems to recur on each decade: Love, know, go, feel ... Words that seem to relate to the popular song that can talk about love. This is consistent with our previous analysis from the pie charts showing the proportions of musical styles across time. We also notice an important presence of onomatopoeia like yeah or oh.

In [35]:
# first decades
tdm = TermsDocumentsMatrix(sdf, decades = [2020],
                          colorscale = 'Plasma')

# most frequent terms
tdm.most_freq_terms(n_rows = 1, n_cols = 1, n_terms = 15)

100%|██████████| 260/260 [00:12<00:00, 21.61it/s]


We get similar results on this second decade with similar high occurrence words. We see a greater amount of onomatopoeia in the current decade. We can explain this by an emergence of the rap music style on this current and last decade. There is in this style of music a very used process, the 'ad-libs'. They are sounds, words or onomatopoeias that the artists pronounce sometimes between two verses or at the end of a sentence to give more impact to their text and to dynamize the atmosphere of a title. This may explain the greater presence of onomatopoeia in the lyrics of this decade.


### Topic modeling with LDA

LDA is a common technic use in topic modeling, we firstly process basic preprocessing.

In [36]:
# gensim
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import datapath

# utils
from datetime import datetime

# dashboards
import pyLDAvis
import pyLDAvis.gensim

# TSNE dependencies
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import numpy as np
import matplotlib.colors as mcolors


class LDATopicModeling():
    
    def __init__(self, df = sdf,
                 decade = 1970,
                 directory = "/kaggle/working/models/",
                 existing = False,
                 n_topics = 10,
                lang_preprocess = preprocess):
        # Apply preprocessing on decade data
        documents = df.loc[df.decade == decade, 'lyrics'].apply(lang_preprocess)
            
        # Create a corpus from a list of texts
        self.__id2word = Dictionary(documents.tolist())
        self.__corpus = [self.__id2word.doc2bow(doc) for doc in documents]
        self.__n_topics = n_topics
        
        #training
        if os.path.isfile(existing):
            # Load a potentially pretrained model from disk.
            self.model = LdaModel.load(temp_file)
        else:
            self.model = LdaModel(corpus=tqdm(self.__corpus),
                                      id2word=self.__id2word,
                                      num_topics=n_topics)
        # directory path
        self.__directory = directory
    
    @property
    def get_id2word(self):
        return self.__id2word
    
    @property
    def get_corpus(self):
        return self.__corpus
    
    def save_current_model(self):
        # retrieve time
        now = datetime.now()
        # create the directory if it doesn't exist
        try:
            os.makedirs(directory + now.strftime("%d%m%Y_%H%M%S"))
        except:
            pass
        # Save model to disk.
        temp_file = datapath(directory + now.strftime("%d%m%Y_%H%M%S") + '/model')
        
        self.model.save(temp_file)
    
    def dashboard_LDAvis(self):
        # some basic dataviz
        pyLDAvis.enable_notebook()
        return pyLDAvis.gensim.prepare(self.model, self.__corpus, self.__id2word)
        
    def plot_tsne(self):
        # n-1 rows each is a vector with i-1 posisitons, where n the number of documents
        # i the topic number and tmp[i] = probability of topic i
        topic_weights = []
        for row_list in self.model[self.get_corpus]:
            tmp = np.zeros(self.__n_topics)
            for i, w in row_list:
                tmp[i] = w
            topic_weights.append(tmp)


        # Array of topic weights    
        arr = pd.DataFrame(topic_weights).fillna(0).values

        # Keep the well separated points (optional)
        arr = arr[np.amax(arr, axis=1) > 0.35]

        # Dominant topic number in each doc
        topic_num = np.argmax(arr, axis=1)
        print("topic_num :", topic_num)

        # tSNE Dimension Reduction
        tsne_model = TSNE(n_components=2, verbose=1, init='pca')
        tsne_lda = tsne_model.fit_transform(arr)

        # Plot the Topic Clusters using Bokeh
        output_notebook()
        mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
        plot = figure(title="t-SNE Clustering of {} LDA Topics".format(self.__n_topics), 
                      plot_width=900, plot_height=700)
        plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
        show(plot)
    

In [37]:
# create my model
lda_model = LDATopicModeling()

100%|██████████| 708/708 [00:00<00:00, 817.40it/s]


In [38]:
# print the result
lda_model.model.print_topics()

[(0,
  '0.024*"time" + 0.018*"know" + 0.013*"get" + 0.010*"la" + 0.009*"love" + 0.007*"let" + 0.007*"feel" + 0.007*"go" + 0.007*"fire" + 0.006*"look"'),
 (1,
  '0.019*"know" + 0.016*"baby" + 0.013*"like" + 0.012*"tell" + 0.011*"get" + 0.011*"love" + 0.011*"go" + 0.010*"say" + 0.010*"time" + 0.009*"night"'),
 (2,
  '0.059*"love" + 0.019*"oh" + 0.019*"know" + 0.018*"go" + 0.013*"yeah" + 0.013*"get" + 0.012*"come" + 0.012*"baby" + 0.011*"want" + 0.010*"away"'),
 (3,
  '0.028*"la" + 0.011*"feel" + 0.011*"people" + 0.011*"leave" + 0.011*"hold" + 0.010*"get" + 0.010*"go" + 0.010*"bad" + 0.009*"na" + 0.009*"whoa"'),
 (4,
  '0.039*"love" + 0.026*"good" + 0.016*"come" + 0.013*"time" + 0.013*"like" + 0.012*"go" + 0.012*"get" + 0.010*"know" + 0.009*"day" + 0.008*"let"'),
 (5,
  '0.027*"la" + 0.021*"love" + 0.019*"man" + 0.018*"go" + 0.017*"to" + 0.016*"holy" + 0.013*"know" + 0.011*"feel" + 0.010*"way" + 0.010*"wear"'),
 (6,
  '0.019*"oh" + 0.013*"know" + 0.012*"yeah" + 0.011*"love" + 0.010*"tell"

In [39]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# pyLDAvis dashboards
#lda_model.dashboard_LDAvis()
pyLDAvis.enable_notebook(local=True)
vis = pyLDAvis.gensim.prepare(lda_model.model,
                        lda_model.get_corpus, 
                        lda_model.get_id2word)
pyLDAvis.display(vis,  local=True)

In [40]:
lda_model.plot_tsne()

topic_num : [8 7 2 8 2 8 8 9 2 2 1 8 5 5 8 7 1 7 3 2 5 7 3 4 9 9 7 7 4 4 7 7 2 9 9 2 1
 9 2 6 2 9 2 9 6 8 2 7 9 8 1 4 3 7 5 9 1 6 6 7 2 1 5 0 2 2 2 9 2 8 2 7 1 3
 9 8 9 2 1 1 2 1 7 0 2 1 4 7 5 2 6 5 2 2 1 8 6 4 8 6 2 6 2 3 4 2 1 4 8 4 0
 7 0 0 6 0 4 8 9 1 2 9 7 7 1 4 4 6 3 0 7 2 7 5 7 2 3 6 9 2 2 2 6 2 5 4 0 6
 9 6 6 2 6 4 1 8 7 7 7 4 6 7 1 7 4 9 8 7 9 9 4 7 1 5 9 9 2 2 6 8 2 4 1 7 6
 3 9 4 4 5 7 2 6 5 8 6 9 9 4 1 0 8 9 6 3 9 0 9 7 7 7 9 0 9 2 9 8 5 9 9 6 9
 8 6 3 6 6 8 0 9 5 2 9 4 6 8 7 8 2 1 7 3 0 0 2 0 0 7 5 5 8 2 9 1 2 1 7 0 3
 9 7 3 5 7 2 7 2 9 6 0 9 3 8 4 2 2 5 9 9 3 2 1 4 7 5 1 0 1 7 2 6 1 2 7 6 8
 8 8 0 7 8 9 8 7 8 1 9 2 2 9 2 7 2 7 8 4 6 7 7 9 0 6 7 3 9 6 4 2 9 1 8 1 2
 6 1 0 1 3 2 7 9 8 9 2 1 1 8 4 2 2 4 0 1 5 9 9 9 8 8 2 4 2 2 1 8 7 7 6 2 2
 8 6 2 2 7 8 4 5 1 7 9 9 4 6 3 8 7 8 8 9 2 1 8 5 4 8 4 9 8 6 7 7 0 8 4 2 2
 0 0 9 2 4 1 9 3 3 2 2 4 6 6 9 7 2 3 6 9 8 9 8 6 7 0 8 9 2 0 4 4 8 2 7 1 1
 2 9 2 1 2 8 9 7 3 3 9 3 3 0 4 8 2 1 0 6 0 6 2 5 8 4 4 8 2 5 2 9 7 9 7 1 0
 1 8 9 0 7 5 

Loading BokehJS ...

In [41]:
lda_model.dashboard_LDAvis()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.040322 -0.030643       1        1  15.682156
1      0.014951  0.013714       2        1  13.367713
9      0.025421  0.014160       3        1  13.358424
7      0.034133  0.009842       4        1  12.294064
8      0.022203 -0.012247       5        1  10.289243
4      0.018677 -0.018257       6        1   9.307391
6      0.032565  0.044310       7        1   7.737509
5     -0.040113 -0.086115       8        1   6.242155
0     -0.049003  0.050221       9        1   6.072261
3     -0.099155  0.015015      10        1   5.649084, topic_info=       Term         Freq        Total Category  logprob  loglift
1308     la   404.000000   404.000000  Default  30.0000  30.0000
44     love  1701.000000  1701.000000  Default  29.0000  29.0000
1885   holy    84.000000    84.000000  Default  28.0000  28.0000
7      good   465.000000   465.000000  Default  27.0000  27.0000
36     feel   467.000000   467.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
0      baby    20.430661   829.507482  Topic10  -5.2468  -0.8301
155    time    20.009985   690.843393  Topic10  -5.2676  -0.6680
41     know    18.317067   998.553918  Topic10  -5.3560  -1.1248
55    thing    17.712287   347.827913  Topic10  -5.3896  -0.1038
16       oh    18.273910   986.320286  Topic10  -5.3584  -1.1148

[771 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1582      1  0.053065       aah
1582      2  0.795976       aah
1582      8  0.053065       aah
1582     10  0.053065       aah
3550      9  0.568664  actually
...     ...       ...       ...
4759     10  0.123380      zion
1264      2  0.760716      zule
1264      3  0.022374      zule
1264      5  0.178992      zule
1264      9  0.022374      zule

[2314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 10, 8, 9, 5, 7, 6, 1, 4])

## 2015 songs lyrics topic modelling

Let's first retrieve the english song in 2015 (year of with max genius lyrics repertoried).

Let's plot the tag distribution

The barplot below shows the frequency of each tag color by total views

Let's try topic modelling with top2vec library which is the easiest to start. But first let's filter the lyrics.

# 2000 song lyrics topic modelling

## References